## 逻辑回归 Python实现

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
### prepare the dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal_l','sepal_w','petal_l','petal_w','label']
df = df[df['label'].apply(lambda x:x in [0,1])]
X = np.array(df.iloc[:,:4])
Y = np.array(df['label'])

In [5]:
df.head()

,sepal_l,sepal_w,petal_l,petal_w,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
class LR():
    def __init__(self, method = 'gd', max_iter = 5000, tol = 0.00001, alpha = 0.1):
        """
        'gd' gradient descent
        'sgd' stochastic gradient descent
        'newton' newton method
        """
        self.method = method
        self.max_iter = max_iter
        self.tol = tol
        self.alpha = alpha
    def _init_parameters(self, features, labels):
        self.n = features.shape[0]
        self.p = features.shape[1]
        self.beta =  np.zeros((self.p+1,1))
        self.X = np.hstack((np.ones((self.n,1)),features))
        self.Y = labels
        
    def _p1(self, beta,feature):
        """
        feature: (p+1) X 1 维  
        """
        return(1/(1+np.exp(-(feature @ beta))))
    
    def _logloss(self, beta1):
        sum1 = 0
        for i in range(self.n):
            sum1 += - self.Y[i] * self.X[i,:] @  beta1 + np.log(1+np.exp(self.X[i,:] @  beta1))
        return(sum1/self.n)
    
    def _train_gd(self, features, labels):
        """
        gradient descent method
        """
        self._init_parameters(features, labels)
        for j in range(self.max_iter):
            beta_now = self.beta
            loss1 = self._logloss(beta_now)
            ### check the log and loss
            if j % 500 ==0:
                print('%d:%f'%(j,loss1))
            r = []
            for i in range(self.n):
                r.append(self._p1(beta_now, self.X[i,:]) - self.Y[i])
            gradient = (1/self.n) * self.X.T @ np.array(r).reshape((self.n,1))
            beta_try = beta_now - self.alpha * gradient
            ct_alpha = 0
            #### make sure the loss go to downhill
            while self._logloss(beta_try) > loss1:
                self.alpha /= 2
                beta_try = beta_now - self.alpha * gradient
                ct_alpha +=1
                if ct_alpha > 100:
                    beta_try = beta_now
                    break
            self.beta = beta_try
            #### stop condition
            if (np.absolute(self.beta - beta_now)).mean() < self.tol:
                break
                
    def _train_sgd(self, features, labels):
        """
        stochastic gradient descent
        """
        self._init_parameters(features, labels)
        self.max_iter = 10000
        self.alpha = 0.001
        loss1 = self._logloss(self.beta)
        for j in range(self.max_iter):
            Ind = list(range(self.n))
            np.random.shuffle(Ind)
            if j % 500 ==0:
                print('%d:%f'%(j,loss1))
            for k in Ind:
                beta_now = self.beta
                loss1 = self._logloss(beta_now)
                gradient = self.X[k,:].reshape((self.p+1,1)) * (self._p1(beta_now, self.X[k,:]) - self.Y[k])
                beta_try = beta_now - self.alpha * gradient
                self.beta = beta_try
                if (np.absolute(self.beta - beta_now)).mean() < self.tol:
                    break
                
    def _train_newton(self, features, labels):
        """
        newton method
        """
        self._init_parameters(features, labels)
        for j in range(50):
            loss1 = self._logloss(self.beta)
            ### check the log and loss
            if j % 2 ==0:
                print('%d:%f'%(j,loss1))
            r = []
            w = []
            for i in range(self.n):
                r.append(self._p1(self.beta, self.X[i,:]) - self.Y[i])
                w.append(np.asscalar(self._p1(self.beta, self.X[i,:]) * (1 - self._p1(self.beta, self.X[i,:]))))
            gradient = (1/self.n) * self.X.T @ np.array(r).reshape((self.n,1))
            hessian = (1/self.n) * self.X.T @ np.diag(w) @ self.X
            beta_now = self.beta
            self.beta = self.beta - np.linalg.inv(hessian) @ gradient
            if (np.absolute(self.beta - beta_now)).mean() < 0.1 or loss1 < 0.0001:
                break
    
    def train(self, features, labels):
        if self.method == 'gd':
            self._train_gd(features, labels)
        if self.method == 'sgd':
            self._train_sgd(features, labels)
        if self.method == 'newton':
            self._train_newton(features, labels)
            
    def predict(self, X_pred):
        X_pred = np.hstack((np.ones((X_pred.shape[0],1)),X_pred))
        sav = []
        for i in range(X_pred.shape[0]):
            p11 = self._p1(self.beta, X_pred[i,:])
            if p11 >= 0.5:
                sav.append(1.0)
            else:
                sav.append(0.0)
        return(np.array(sav))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 71)

#### 批量梯度下降法

In [8]:
Logistic = LR()
Logistic.train(X_train,y_train)
Logistic.predict(X_test) == y_test

0:0.693147
500:0.013442
1000:0.006936
1500:0.004715
2000:0.003587
2500:0.002902
3000:0.002441
3500:0.002109
4000:0.001858
4500:0.001661


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [9]:
Logistic.beta

array([[-0.46585927],
       [-0.73805583],
       [-2.68092378],
       [ 4.02513816],
       [ 1.76894571]])

#### 牛顿迭代法

In [10]:
Logistic_n = LR(method='newton')
Logistic_n.train(X_train,y_train)
Logistic_n.predict(X_test) == y_test

0:0.693147
2:0.050291
4:0.007132
6:0.001031
8:0.000148


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [11]:
Logistic_n.beta

array([[ 3.81918051],
       [-1.79434612],
       [-4.94152943],
       [ 5.4720225 ],
       [ 7.25666619]])

#### 随机梯度下降

In [12]:
Logistic_s = LR(method='sgd')
Logistic_s.train(X_train,y_train)
Logistic_s.predict(X_test) == y_test

0:0.693147
500:0.023867
1000:0.019200
1500:0.016948
2000:0.015531
2500:0.014398
3000:0.013462
3500:0.012585
4000:0.012018
4500:0.011469
5000:0.010964
5500:0.010555
6000:0.010232
6500:0.009950
7000:0.009675
7500:0.009430
8000:0.009210
8500:0.009014
9000:0.008832
9500:0.008662


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [14]:
Logistic_s.beta

array([[-0.33330666],
       [-0.52684949],
       [-1.91391143],
       [ 2.86784246],
       [ 1.24277164]])

#### Sklearn包

In [25]:
LRSK = LogisticRegression()
LRSK.fit(X_train,y_train)
LRSK.predict(X_test) == y_test

E:\software\ANACONDA3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [26]:
LRSK.intercept_

array([-0.24002507])

In [27]:
LRSK.coef_

array([[-0.38039368, -1.35538395,  2.02958925,  0.88941473]])

### 总结
- 从预测正确率来说，三种方法都和Skleran包一样对测试集达到了100%的预测，说明该程序还是有效的,应该没有什么重大的bug。、
- 从loss下降角度来说，三种方法的loss都在下降，也侧面说明了程序无重大bug。另外可以明显的看出牛顿迭代的loss下降明显比其他两者快得多，另外因为这里数据量较少，没能体现出随机梯度下降和批量梯度下降的差距。
- 另外，观察四种方法得到的超平面的参数，似乎都不太一样。首先可能是由于每种方法得出的beta的scale不一样，导致所有的看去都不太相同。另外由于该数据可能接近线性可分，所以能使其loss最低的超平面不唯一。 最后，sklearn包中有正则化过程，我写的里面没有，所以这也是原因之一。